In [154]:
import torch
import random
import numpy as np
import pandas as pd

random.seed(0)#номер случайной последовательности, которую выдает генератор случайных чисел
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True#для воспроизводимости эксперимента


In [155]:
import csv
with open("mushrooms.csv", encoding='utf-8') as r_file:
    # Создаем объект reader, указываем символ-разделитель ","
    data = csv.reader(r_file, delimiter = ",")
    # Счетчик для подсчета количества строк и вывода заголовков столбцов
df = pd.read_csv('mushrooms.csv',  # Это то, куда вы скачали файл
                       sep=',')
df['class'] = df['class'].replace({'e':1,'p':0})

In [156]:
X.head(10)
Y_train = np.array(df['class']) #labels
X_train = df.iloc[:,1:] #data



In [157]:
X_train=pd.get_dummies(X,columns=X.columns,drop_first=True)
X.head(10)

,cap-shape_c,cap-shape_f,cap-shape_k,cap-shape_s,cap-shape_x,cap-surface_g,cap-surface_s,cap-surface_y,cap-color_c,cap-color_e,...,population_n,population_s,population_v,population_y,habitat_g,habitat_l,habitat_m,habitat_p,habitat_u,habitat_w
0,0,0,0,0,1,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
1,0,0,0,0,1,0,1,0,0,0,...,1,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,1,0,0,0
3,0,0,0,0,1,0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
4,0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
5,0,0,0,0,1,0,0,1,0,0,...,1,0,0,0,1,0,0,0,0,0
6,0,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,1,0,0,0
7,0,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,1,0,0,0
8,0,0,0,0,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
9,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,1,0,0,0


In [158]:
y_train = torch.tensor(df['class']).float() #labels
Y_train = y_train[:7624]
x_train = torch.tensor(X_train.values[:7624,:])
X_test =torch.tensor(X_train.values[7624:,:])
y_test = y_train[7624:]
print(len(X_test[0]))

95


In [149]:
class WineNet(torch.nn.Module):
    def __init__(self,n_input, n_hidden_neurons):
        super(WineNet,self).__init__()
        #типикал структура сети,интересно почему 2 входа, но это еще прокомменчу
        self.fc1 = torch.nn.Linear(95,n_hidden_neurons)#2 входа, ибо 2 колонки вина
        self.activ1 = torch.nn.Sigmoid()
        self.fc2 = torch.nn.Linear(n_hidden_neurons,n_hidden_neurons)
        self.activ2 = torch.nn.Sigmoid()
        self.fc3 = torch.nn.Linear(n_hidden_neurons,2)#3 возможных класса  => 3 возможных выхода
        self.sm = torch.nn.Softmax(dim = 1)#софт-макс, принимающий на вход одномерный тензор
        
    def forward(self,x):
        x = self.fc1(x)
        x = self.activ1(x)
        x = self.fc2(x)
        x = self.activ2(x)
        x = self.fc3(x)#хде софт-макс??
        return x
#выписываешь формулы кросс-энтропии и софт-макса и понимаешь что в одной экспа, в другой лог и все  уходит и софт-макс нахуй не нужен
#для лосс-фукнции софт-макс нахуй не сдался, а вот для вероятностей пригодится    
    def inference(self,x):#cчитаю вероятности выпадения определенного сорта говна
        x = self.forward(x)
        x = self.sm(x)
        return x
n_input = 95
n_hidden = 180
wine_net = WineNet(n_input, n_hidden)
#мы стараемся избегать софт-макса, ибо он весьма долгий

In [150]:
loss = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(wine_net.parameters(), 
                             lr=1.0e-3) 

In [153]:
batch_size = 10

for epoch in range(5000):
    
    order = np.random.permutation(len(x_train))
   
    for start_index in range(0, len(x_train), batch_size):
    #каждую эпоху мешаю индексы и определяю в каком порядке буду их вытаскивать из сета
        optimizer.zero_grad()#зануляю градиенты
        batch_indexes = order[start_index:start_index+batch_size]
    #каждую эпоху вырезаю куски из сета длинной в бач
    
        x_batch = x_train[batch_indexes]#вытаскиваю куски из сета
        y_batch = y_train[batch_indexes]
        preds = wine_net.forward(x_batch.float())#сделаю небольшие предикты по бачам, по 3 выходам из сети
    
        loss_val = loss(preds, y_batch.long())#считаю лоссы по предиктам и лейблам
    
        loss_val.backward()#cчитаю производную по лоссу
    
        optimizer.step() # а теперь загоняю всю эту хуету в оптимайзер и делаю шаг гсп
    #он как бы оборачивает собой все веса сети
    
    if epoch %10 == 0:
        #каждые 100 эпох чекаем метрики для проверки тупости сети
        test_preds = wine_net.forward(X_test[epoch//10].float())
       
        test_preds = test_preds.argmax(dim = 0)#чекаем какой класс предсказывает сеть
        #сравниваем номер нейрона в выходе и сравниваем его лейблом из теста
        
        print(wine_net.fc1.in_features, np.asarray((test_preds == y_test).float().mean()) > 0.8)
        #смотрим насколько сильно они совпадают
        #тк мы не можем посчитать вероятность целочисленного, мы его приводим к дробному и считаем

95 False
95 False


KeyboardInterrupt: 